# Σημασιολογικός Πυρήνας με Ενσωμάτωση OpenBnB MCP Server

Αυτό το σημειωματάριο δείχνει πώς να χρησιμοποιήσετε τον Σημασιολογικό Πυρήνα με τον πραγματικό OpenBnB MCP server για να αναζητήσετε πραγματικά καταλύματα Airbnb χρησιμοποιώντας το MCPStdioPlugin. Για πρόσβαση στο LLM, χρησιμοποιεί το Azure AI Foundry. Για να ρυθμίσετε τις μεταβλητές περιβάλλοντος, μπορείτε να ακολουθήσετε το [Μάθημα Ρύθμισης](/00-course-setup/README.md)


## Εισαγωγή των Απαραίτητων Πακέτων


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Δημιουργία Σύνδεσης MCP Plugin

Θα συνδεθούμε με τον [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) χρησιμοποιώντας το MCPStdioPlugin. Αυτός ο διακομιστής παρέχει λειτουργικότητα αναζήτησης Airbnb μέσω του πακέτου @openbnb/mcp-server-airbnb.


## Δημιουργία του Πελάτη

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε το Azure AI Foundry για την πρόσβαση στο LLM. Βεβαιωθείτε ότι οι μεταβλητές περιβάλλοντος έχουν ρυθμιστεί σωστά.


## Ρύθμιση Περιβάλλοντος

Ρυθμίστε τις παραμέτρους του Azure OpenAI. Βεβαιωθείτε ότι έχετε ορίσει τις παρακάτω μεταβλητές περιβάλλοντος:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Κατανόηση της Ενσωμάτωσης OpenBnB MCP

Αυτό το σημειωματάριο συνδέεται με τον **πραγματικό OpenBnB MCP server**, ο οποίος παρέχει λειτουργικότητα αναζήτησης Airbnb.

### Πώς λειτουργεί:

1. **MCPStdioPlugin**: Χρησιμοποιεί επικοινωνία μέσω τυπικής εισόδου/εξόδου με τον MCP server
2. **Πραγματικό NPM Πακέτο**: Κατεβάζει και εκτελεί το `@openbnb/mcp-server-airbnb` μέσω npx
3. **Ζωντανά Δεδομένα**: Επιστρέφει πραγματικά δεδομένα ιδιοκτησιών Airbnb από τα APIs τους
4. **Ανακάλυψη Λειτουργιών**: Ο agent ανακαλύπτει αυτόματα τις διαθέσιμες λειτουργίες από τον MCP server

### Διαθέσιμες Λειτουργίες:

Ο OpenBnB MCP server συνήθως παρέχει:
- **search_listings** - Αναζήτηση ιδιοκτησιών Airbnb βάσει τοποθεσίας και κριτηρίων
- **get_listing_details** - Λήψη λεπτομερών πληροφοριών για συγκεκριμένες ιδιοκτησίες
- **check_availability** - Έλεγχος διαθεσιμότητας για συγκεκριμένες ημερομηνίες
- **get_reviews** - Ανάκτηση κριτικών για ιδιοκτησίες
- **get_host_info** - Λήψη πληροφοριών για τους οικοδεσπότες ιδιοκτησιών

### Προαπαιτούμενα:

- Εγκατεστημένο **Node.js** στο σύστημά σας
- **Σύνδεση στο Διαδίκτυο** για τη λήψη του πακέτου MCP server
- **NPX** διαθέσιμο (περιλαμβάνεται στο Node.js)

### Δοκιμή της Σύνδεσης:

Μπορείτε να δοκιμάσετε τον MCP server χειροκίνητα εκτελώντας:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Αυτό θα κατεβάσει και θα ξεκινήσει τον OpenBnB MCP server, στον οποίο στη συνέχεια συνδέεται το Semantic Kernel για πραγματικά δεδομένα Airbnb.


## Εκτέλεση του Agent με τον OpenBnB MCP Server

Τώρα θα εκτελέσουμε τον AI Agent που συνδέεται με τον OpenBnB MCP server για να αναζητήσει πραγματικά καταλύματα Airbnb στη Στοκχόλμη για 2 ενήλικες και 1 παιδί. Μπορείτε να τροποποιήσετε τη λίστα `user_inputs` για να αλλάξετε τα κριτήρια αναζήτησης.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Περίληψη
Συγχαρητήρια! Έχετε δημιουργήσει με επιτυχία έναν AI agent που ενσωματώνεται με την αναζήτηση καταλυμάτων στον πραγματικό κόσμο χρησιμοποιώντας το Model Context Protocol (MCP):

## Τεχνολογίες που Χρησιμοποιήθηκαν:
- Semantic Kernel - Για τη δημιουργία έξυπνων agents με το Azure OpenAI
- Azure AI Foundry - Για δυνατότητες LLM και ολοκλήρωση συνομιλιών
- MCP (Model Context Protocol) - Για τυποποιημένη ενσωμάτωση εργαλείων
- OpenBnB MCP Server - Για πραγματική λειτουργικότητα αναζήτησης Airbnb
- Node.js/NPX - Για την εκτέλεση του εξωτερικού MCP server

## Τι Έχετε Μάθει:
- Ενσωμάτωση MCP: Σύνδεση των agents του Semantic Kernel με εξωτερικούς MCP servers
- Πρόσβαση σε Δεδομένα σε Πραγματικό Χρόνο: Αναζήτηση πραγματικών καταλυμάτων Airbnb μέσω ζωντανών APIs
- Επικοινωνία Πρωτοκόλλου: Χρήση επικοινωνίας stdio μεταξύ του agent και του MCP server
- Ανακάλυψη Λειτουργιών: Αυτόματη ανακάλυψη διαθέσιμων λειτουργιών από MCP servers
- Ροές Απαντήσεων: Καταγραφή και καταγραφή κλήσεων λειτουργιών σε πραγματικό χρόνο
- Απόδοση HTML: Μορφοποίηση των απαντήσεων του agent με στυλιζαρισμένους πίνακες και διαδραστικές προβολές

## Επόμενα Βήματα:
- Ενσωμάτωση επιπλέον MCP servers (καιρός, πτήσεις, εστιατόρια)
- Δημιουργία συστήματος πολλαπλών agents που συνδυάζει MCP και A2A πρωτόκολλα
- Δημιουργία προσαρμοσμένων MCP servers για τις δικές σας πηγές δεδομένων
- Υλοποίηση μνήμης συνομιλίας που παραμένει μεταξύ των συνεδριών
- Ανάπτυξη του agent σε Azure Functions με ορχήστρωση MCP server
- Προσθήκη δυνατότητας αυθεντικοποίησης χρηστών και κρατήσεων



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
